In [11]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os

%matplotlib inline

In [21]:
file_paths = {
    "Passengers": r"E:\数据集\ShipEar\data_preprocessing\4_Frame_Windows_3s_0%\10__10_07_13_marDeOnza_Sale_26.wav",
    "Tugboat":r"E:\数据集\ShipEar\data_preprocessing\4_Frame_Windows_3s_0%\15__10_07_13_radaUno_Pasa_26.wav",
    "RORO":r"E:\数据集\ShipEar\data_preprocessing\4_Frame_Windows_3s_0%\58__23_07_13_H2_EimskipReefer_Pesquerito1_58.wav",
    "Motorboat":r"E:\数据集\ShipEar\data_preprocessing\4_Frame_Windows_3s_0%\77__23_07_13_H3_Planeadora_28.wav",
    "Fishboat":r"E:\数据集\ShipEar\data_preprocessing\4_Frame_Windows_3s_0%\76__23_07_13_H3_pesquero2_28.wav",
    "Noise":r"E:\数据集\ShipEar\data_preprocessing\4_Frame_Windows_3s_0%\82__27_09_13_H3_lluvia_18.wav"
}

SR = 16000
N_FFT = 2048
HOP_LENGTH = 512
N_MELS = 128
N_MFCC = 40
FMAX = SR // 2
OUTPUT_DIR = r"E:\BaiduSyncdisk\文献综述、开题报告\绘图" 
OUTPUT_FORMAT = "png"
OUTPUT_DPI = 600

In [17]:
def save_individual_mels(file_paths, output_dir):
    
    print("开始逐个处理并保存图像...")
    
    for i, (class_name, file_path) in enumerate(file_paths.items()):
        # ================= 核心变化 =================
        # 在循环内部为每一张图创建一个新的、独立的画布
        # figsize=(4, 3) 是一个适合单张论文插图的通用比例
        fig, ax = plt.subplots(figsize=(4, 3))
        
        # 彻底关闭坐标轴，确保无边框无刻度，只留干货
        ax.axis('off') 
        # ==========================================
        
        try:
            print(f"[{i+1}/{len(file_paths)}] 正在处理: {class_name} ...")
            y, sr = librosa.load(file_path, sr=SR)
            S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=N_FFT, 
                                               hop_length=HOP_LENGTH, 
                                               n_mels=N_MELS, fmax=FMAX)
            S_dB = librosa.power_to_db(S, ref=np.max)
            
            # 绘制
            # rasterized=True 对PNG没用，但对PDF有用，留着也无妨
            librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', 
                                     sr=sr, fmax=FMAX, ax=ax, cmap='magma')
            
            # 构建独立的文件名
            # 例如: paper_figures_individual/mel_Passengers.png
            save_name = f"mel_{class_name}.{OUTPUT_FORMAT}"
            full_save_path = os.path.join(output_dir, save_name)
            
            # 保存高质量图像
            # bbox_inches='tight', pad_inches=0 保证了去除所有白边
            plt.savefig(full_save_path, 
                        format=OUTPUT_FORMAT, 
                        dpi=OUTPUT_DPI, 
                        bbox_inches='tight', 
                        pad_inches=0)
            
        except Exception as e:
            print(f"  Error processing {class_name}: {e}")
            # 如果出错，保存一张带有错误提示的图，避免流程中断
            ax.text(0.5, 0.5, f"ERROR: {class_name}", 
                    ha='center', va='center', color='red', transform=ax.transAxes)
            plt.savefig(os.path.join(output_dir, f"ERROR_{class_name}.png"))

        finally:
            # ================= 关键一步 =================
            # 保存后立即关闭当前画布，释放内存，防止图像重叠
            # 这样 Notebook 里就不会显示这一堆图了，它们直接进了文件夹
            plt.close(fig)
            # ==========================================

    print("\n所有图像处理完成！请查看输出目录。")

In [18]:
save_individual_mels(file_paths, OUTPUT_DIR)

开始逐个处理并保存图像...
[1/6] 正在处理: Passengers ...
[2/6] 正在处理: Tugboat ...
[3/6] 正在处理: RORO ...
[4/6] 正在处理: Motorboat ...
[5/6] 正在处理: Fishboat ...
[6/6] 正在处理: Noise ...

所有图像处理完成！请查看输出目录。


In [23]:
def save_individual_mfccs(file_paths, output_dir):
    
    print(f"开始计算并保存 {N_MFCC} 阶 MFCC 图像...")
    
    for i, (class_name, file_path) in enumerate(file_paths.items()):
        # 创建独立画布，保持适合论文插图的比例
        fig, ax = plt.subplots(figsize=(4, 3))
        
        # 彻底关闭坐标轴，只留干货
        ax.axis('off') 
        
        try:
            print(f"[{i+1}/{len(file_paths)}] 正在处理: {class_name} ...")
            y, sr = librosa.load(file_path, sr=SR)
            
            # ================= MFCC 计算核心 =================
            # librosa.feature.mfcc 会自动先计算 Mel 谱图，再进行离散余弦变换(DCT)
            # n_mfcc 决定了 Y 轴有多少行
            mfcc_data = librosa.feature.mfcc(y=y, sr=sr, n_fft=N_FFT, 
                                             hop_length=HOP_LENGTH, 
                                             n_mels=N_MELS, fmax=FMAX, 
                                             n_mfcc=N_MFCC)
            
            # 可选：对数据进行标准化，有时能让视觉对比度更好
            # mfcc_data = sklearn.preprocessing.scale(mfcc_data, axis=1)
            # 这里暂时保持原始数据，通常 magma 色图也能很好地展示
            # ================================================
            
            # 绘制
            # 注意：对于 MFCC，y_axis 不需要指定为 'mel'，默认即可显示系数索引
            # x_axis='time' 仍然适用
            # 使用 'magma' 或 'viridis' 这种感知均匀的色图对于有正有负的数据效果不错
            librosa.display.specshow(mfcc_data, x_axis='time', 
                                     sr=sr, hop_length=HOP_LENGTH, 
                                     ax=ax, cmap='coolwarm')
            
            # 构建文件名
            save_name = f"mfcc_{class_name}.{OUTPUT_FORMAT}"
            full_save_path = os.path.join(output_dir, save_name)
            
            # 保存高质量图像
            plt.savefig(full_save_path, 
                        format=OUTPUT_FORMAT, 
                        dpi=OUTPUT_DPI, 
                        bbox_inches='tight', 
                        pad_inches=0)
            
        except Exception as e:
            print(f"  Error processing {class_name}: {e}")
            ax.text(0.5, 0.5, f"ERROR: {class_name}", 
                    ha='center', va='center', color='red', transform=ax.transAxes)
            plt.savefig(os.path.join(output_dir, f"ERROR_MFCC_{class_name}.png"))

        finally:
            # 关闭画布，释放内存
            plt.close(fig)

    print("\n所有 MFCC 图像处理完成！请查看输出目录。")

In [24]:
save_individual_mfccs(file_paths, OUTPUT_DIR)

开始计算并保存 40 阶 MFCC 图像...
[1/6] 正在处理: Passengers ...
[2/6] 正在处理: Tugboat ...
[3/6] 正在处理: RORO ...
[4/6] 正在处理: Motorboat ...
[5/6] 正在处理: Fishboat ...
[6/6] 正在处理: Noise ...

所有 MFCC 图像处理完成！请查看输出目录。
